**1. Optimization practice** this is just a review of the code we have for solving Bellman equations. 

In [ ]:

function expectation(v, P)
    return v*P'
end

function T_operator(v,kgrid,agrid,param,P,Ri) 
    """
    Defines the T operator as the list of operations to solve the maximization program
    in the right hand side of the Bellman equation: max u(c_t)+β v(k_{t+1})
    
    Input:
      v:     The guess of the value function at current iteration 
      kgrid: The capital grid
      param: A (4x1) vector of parameters
      agrid: Grid of values for the shock
      P    : Transition matrix for the shock
    
    Output: 
     Tv: The new guess for the value function (we applied the T operator to v)
     Gk: The implied optimal capital decision rule
    """   


    α, β, δ, σ = param 
    Tv= size(v)
    Gk= size(v) ##these we just to be the same as the structure of state variables we may have. 
    Nk,Na= size(v) #this facilitates looping over stuff. Here we just need an outer and and inner loop to work. On both state variables. 
    Ev= expectation(v,P)

    for ia in 1:size(Na)
        next_index=1
            for ik in 1:size(Nk)
                vmax=-Inf
                k_prime=0.0
                r=Ri(ia, ik)
                    for jk in next_index:Nk
                        kp=kgrid[jk]
                        vj=util(r-kp,σ)+ β*Ev(ia, jk)
                        if vj>vmax 
                            next_index=jk
                            vmax=vj
                            k_prime=kp
                        else 
                            break
                        end 
                    end 
                @inbounds Tv[ia,ik]= vmax
                @inbounds Gk[ia, ik]= k_prime
                end
            end 
        end


T_operator (generic function with 1 method)